In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import math
import pickle
import random
import pickle
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer

In [ ]:
def to_y(argument):
    switcher = {
        'c0':[1,0,0,0,0,0,0,0,0,0],
        'c1':[0,1,0,0,0,0,0,0,0,0],
        'c2':[0,0,1,0,0,0,0,0,0,0],
        'c3':[0,0,0,1,0,0,0,0,0,0],
        'c4':[0,0,0,0,1,0,0,0,0,0],
        'c5':[0,0,0,0,0,1,0,0,0,0],
        'c6':[0,0,0,0,0,0,1,0,0,0],
        'c7':[0,0,0,0,0,0,0,1,0,0],
        'c8':[0,0,0,0,0,0,0,0,1,0],
        'c9':[0,0,0,0,0,0,0,0,0,1],
    }
    return switcher.get(argument, "nothing")

In [ ]:
reader=pd.read_csv('data.csv')
y0=[]
for i in reader['classname']:
    y0.append(to_y(i))
data=[]
import random
sclice = random.sample(range(len(yy)), len(yy))
for i in sclice:
    data.append([y0[i],'train/'+reader['img'][i]])

In [ ]:
n_batches = 40
batch_size = 512 
test_features=[]
test_labels=[]
for i in range((n_batches+1)*batch_size,len(y0)):
    img = cv2.imread(data[i][1])
    resized = cv2.resize(img, (299,299), interpolation=cv2.INTER_AREA)
    test_features.append(resized)
    test_labels.append(data[i][0])

In [ ]:
def neural_net_image_input(image_shape):
    """
    Return a Tensor for a batch of image input
    : image_shape: Shape of the images
    : return: Tensor for image input.
    """
    i =tf.placeholder(
    tf.float32,
    shape=[None, *image_shape], name='x')
    return i


def neural_net_label_input(n_classes):
    """
    Return a Tensor for a batch of label input
    : n_classes: Number of classes
    : return: Tensor for label input.
    """
    i =tf.placeholder(
    tf.float32,
    shape=[None, n_classes], name='y')
    return i

def load_preprocess_training_batch(batch_id, batch_size):
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    features=[]
    labels=[]
    for i in range(batch_id*batch_size,batch_id*batch_size+batch_size):
        img = cv2.imread(data[i][1])
        resized = cv2.resize(img, (299,299), interpolation=cv2.INTER_AREA)
        features.append(resized)
        labels.append(data[i][0])

    # Return the training data in batches of size <batch_size> or less
    yield features, labels
    
def print_stats(session, feature_batch, label_batch, cost, accuracy):
    """
    Print information about loss and validation accuracy
    : session: Current TensorFlow session
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    : cost: TensorFlow cost function
    : accuracy: TensorFlow accuracy function
    """
    loss = session.run(cost, feed_dict={
        x: feature_batch,
        y: label_batch})
    valid_acc = session.run(accuracy, feed_dict={
        x: test_features,
        y: test_labels})
    print(loss,end=' ')
    print(valid_acc)
    return([loss,valid_acc])


zhfont1 = matplotlib.font_manager.FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=16)  

def _load_label_names():
    """
    Load the label names from file
    """
    return [
    u'c0',
    u'c1',
    u'c2',
    u'c3',
    u'c4',
    u'c5',
    u'c6',
    u'c7',
    u'c8',
    u'c9'
]


def display_image_predictions(features, labels, predictions):
    n_classes = 10
    label_names = _load_label_names()
    label_binarizer = LabelBinarizer()
    label_binarizer.fit(range(n_classes))
    label_ids = label_binarizer.inverse_transform(np.array(labels))

    fig, axies = plt.subplots(nrows=4, ncols=2, figsize=(20,12))
    fig.tight_layout()
    fig.suptitle('Predictions', fontsize=40, y=1.1)
    n_predictions = 3
    margin = 0.05
    ind = np.arange(n_predictions)
    width = (1. - 2. * margin) / n_predictions

    for image_i, (feature, label_id, pred_indicies, pred_values) in enumerate(zip(features, label_ids, predictions.indices, predictions.values)):
        pred_names = [label_names[pred_i] for pred_i in pred_indicies]
        correct_name = label_names[label_id]

        axies[image_i][0].imshow(feature)
        axies[image_i][0].set_title(correct_name,fontproperties=zhfont1)
        axies[image_i][0].set_axis_off()

        axies[image_i][1].barh(ind + margin, pred_values[::-1], width)
        axies[image_i][1].set_yticks(ind + margin)
        axies[image_i][1].set_yticklabels(pred_names[::-1],fontproperties=zhfont1)
        axies[image_i][1].set_xticks([0])
  
def batch_features_labels(features, labels, batch_size):
    """
    Split features and labels into batches
    """
    for start in range(0, len(features), batch_size):
        end = min(start + batch_size, len(features))
        yield features[start:end], labels[start:end]
        

def py_returnCAMmap(activation, weights_LR):
#     print(activation.shape)
#     print(weights_LR.shape)
    n_feat, w, h, n= activation.shape
    act_vec = np.reshape(activation, [n_feat, w*h])
    n_top = weights_LR.shape[0]
    out = np.zeros([w, h, n_top])

    for t in range(n_top):
        weights_vec = np.reshape(weights_LR[t], [1, weights_LR[t].shape[0]])
        heatmap_vec = np.dot(weights_vec,act_vec)
        heatmap = np.reshape( np.squeeze(heatmap_vec) , [w, h])
        out[:,:,t] = heatmap

    return out


def im2double(im):
    return cv2.normalize(im.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)


def py_map2jpg(imgmap, rang, colorMap):
    if rang is None:
        rang = [np.min(imgmap), np.max(imgmap)]

    heatmap_x = np.round(imgmap*255).astype(np.uint8)

    return cv2.applyColorMap(heatmap_x, cv2.COLORMAP_JET)

In [ ]:
# coding:UTF-8
slim = tf.contrib.slim
trunc_normal = lambda stddev: tf.truncated_normal_initializer(0.0, stddev)

# 用来生成网络中经常用到的函数的默认参数
# 默认参数：卷积的激活函数、权重初始化方式、标准化器等
def inception_v3_arg_scope(weight_decay=0.00004,    # L2正则的weight_decay
                           stddev=0.1,  # 标准差0.1
                           batch_norm_var_collection='moving_vars'):

    batch_norm_params = {  # 定义batch normalization参数字典
      'decay': 0.9997,  #衰减系数
      'epsilon': 0.001,
      'updates_collections': tf.GraphKeys.UPDATE_OPS,
      'variables_collections': {
          'beta': None,
          'gamma': None,
          'moving_mean': [batch_norm_var_collection],
          'moving_variance': [batch_norm_var_collection],
      }
  }

  # silm.arg_scope可以给函数自动赋予某些默认值
  # 会对[slim.conv2d, slim.fully_connected]这两个函数的参数自动赋值,
  # 使用slim.arg_scope后就不需要每次都重复设置参数了，只需要在有修改时设置
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                      weights_regularizer=slim.l2_regularizer(weight_decay)): # 对[slim.conv2d, slim.fully_connected]自动赋值

      # 嵌套一个slim.arg_scope对卷积层生成函数slim.conv2d的几个参数赋予默认值
        with slim.arg_scope(
            [slim.conv2d],
            weights_initializer=trunc_normal(stddev), # 权重初始化器
            activation_fn=tf.nn.relu, # 激活函数
            normalizer_fn=slim.batch_norm, # 标准化器
            normalizer_params=batch_norm_params) as sc: # 标准化器的参数设置为前面定义的batch_norm_params
            return sc # 最后返回定义好的scope




# 生成V3网络的卷积部分
def inception_v3_base(inputs, scope=None):
    '''
    Args:
    inputs：输入的tensor
    scope：包含了函数默认参数的环境
    '''
    end_points = {} # 定义一个字典表保存某些关键节点供之后使用

    with tf.variable_scope(scope, 'InceptionV3', [inputs]):
        with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d], # 对三个参数设置默认值
                        stride=1, padding='VALID'):

      #  因为使用了slim以及slim.arg_scope，我们一行代码就可以定义好一个卷积层
      #  相比AlexNet使用好几行代码定义一个卷积层，或是VGGNet中专门写一个函数定义卷积层，都更加方便
      #
      # 正式定义Inception V3的网络结构。首先是前面的非Inception Module的卷积层
      # slim.conv2d函数第一个参数为输入的tensor，第二个是输出的通道数，卷积核尺寸，步长stride，padding模式

      #一共有5个卷积层，2个池化层，实现了对图片数据的尺寸压缩，并对图片特征进行了抽象
      # 299 x 299 x 3
#             print(inputs)
            net = slim.conv2d(inputs, 32, [3, 3],
                            stride=2, scope='Conv2d_1a_3x3')    # 149 x 149 x 32

            net = slim.conv2d(net, 32, [3, 3],
                            scope='Conv2d_2a_3x3')      # 147 x 147 x 32

            net = slim.conv2d(net, 64, [3, 3], padding='SAME',
                            scope='Conv2d_2b_3x3')  # 147 x 147 x 64

            net = slim.max_pool2d(net, [3, 3], stride=2,
                                scope='MaxPool_3a_3x3')   # 73 x 73 x 64

            net = slim.conv2d(net, 80, [1, 1],
                            scope='Conv2d_3b_1x1')  # 73 x 73 x 80

            net = slim.conv2d(net, 192, [3, 3],
                            scope='Conv2d_4a_3x3')  # 71 x 71 x 192

            net = slim.max_pool2d(net, [3, 3], stride=2,
                                scope='MaxPool_5a_3x3') # 35 x 35 x 192


    '''
    三个连续的Inception模块组，三个Inception模块组中各自分别有多个Inception Module，这部分是Inception Module V3
    的精华所在。每个Inception模块组内部的几个Inception Mdoule结构非常相似，但是存在一些细节的不同
    '''
    # Inception blocks
    with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d], # 设置所有模块组的默认参数
                        stride=1, padding='SAME'): # 将所有卷积层、最大池化、平均池化层步长都设置为1
      # 第一个模块组包含了三个结构类似的Inception Module
        '''    
        --------------------------------------------------------    
        第一个Inception组   一共三个Inception模块
        '''
        with tf.variable_scope('Mixed_5b'): # 第一个Inception Module名称。Inception Module有四个分支
        # 第一个分支64通道的1*1卷积
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1') # 35x35x64
        
            # 第二个分支48通道1*1卷积，链接一个64通道的5*5卷积
            with tf.variable_scope('Branch_1'):
                branch_1 = slim.conv2d(net, 48, [1, 1], scope='Conv2d_0a_1x1') # 35x35x48
                branch_1 = slim.conv2d(branch_1, 64, [5, 5], scope='Conv2d_0b_5x5') #35x35x64

            # 第三个分支64通道1*1卷积,96的3*3,再接一个3*3
            with tf.variable_scope('Branch_2'):
                branch_2 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
                branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0b_3x3')
                branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0c_3x3')#35x35x96

            # 第四个分支64通道3*3平均池化,32的1*1
            with tf.variable_scope('Branch_3'):
                branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')            
                branch_3 = slim.conv2d(branch_3, 32, [1, 1], scope='Conv2d_0b_1x1') #35*35*32
            net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3) # 将四个分支的输出合并在一起（第三个维度合并，即输出通道上合并）
        # 64+64+96+32 = 256
        # mixed_1: 35 x 35 x 256.
        '''
        因为这里所有层步长均为1，并且padding模式为SAME，所以图片尺寸不会缩小，但是通道数增加了。四个分支通道数之和
        64+64+96+32=256，最终输出的tensor的图片尺寸为35*35*256
        '''

        with tf.variable_scope('Mixed_5c'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch_1 = slim.conv2d(net, 48, [1, 1], scope='Conv2d_0b_1x1')
                branch_1 = slim.conv2d(branch_1, 64, [5, 5], scope='Conv_1_0c_5x5')
            with tf.variable_scope('Branch_2'):
                branch_2 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
                branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0b_3x3')
                branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0c_3x3')
            with tf.variable_scope('Branch_3'):
                branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
                branch_3 = slim.conv2d(branch_3, 64, [1, 1], scope='Conv2d_0b_1x1')
            net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
          # 64+64+96+64 = 288
          # mixed_2: 35 x 35 x 288.

        with tf.variable_scope('Mixed_5d'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch_1 = slim.conv2d(net, 48, [1, 1], scope='Conv2d_0a_1x1')
                branch_1 = slim.conv2d(branch_1, 64, [5, 5], scope='Conv2d_0b_5x5')
            with tf.variable_scope('Branch_2'):
                branch_2 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1')
                branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0b_3x3')
                branch_2 = slim.conv2d(branch_2, 96, [3, 3], scope='Conv2d_0c_3x3')
            with tf.variable_scope('Branch_3'):
                branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
                branch_3 = slim.conv2d(branch_3, 64, [1, 1], scope='Conv2d_0b_1x1')
            net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)

          # 64+64+96+64 = 288
          # mixed_1: 35 x 35 x 288

        '''    
        第一个Inception组结束  一共三个Inception模块 输出为:35*35*288
----------------------------------------------------------------------    
        第二个Inception组   共5个Inception模块
        '''

        with tf.variable_scope('Mixed_6a'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 384, [3, 3], stride=2,
                                     padding='VALID', scope='Conv2d_1a_1x1') #17*17*384
            with tf.variable_scope('Branch_1'):
                branch_1 = slim.conv2d(net, 64, [1, 1], scope='Conv2d_0a_1x1') #35*35*64
                branch_1 = slim.conv2d(branch_1, 96, [3, 3], scope='Conv2d_0b_3x3')#35*35*96
                branch_1 = slim.conv2d(branch_1, 96, [3, 3], stride=2,
                                     padding='VALID', scope='Conv2d_1a_1x1') #17*17*96
            with tf.variable_scope('Branch_2'):
                branch_2 = slim.max_pool2d(net, [3, 3], stride=2, padding='VALID',
                                         scope='MaxPool_1a_3x3') #17*17*288
            net = tf.concat([branch_0, branch_1, branch_2], 3) # 输出尺寸定格在17 x 17 x 768
        # 384+96+288 = 768
        # mixed_3: 17 x 17 x 768.

        with tf.variable_scope('Mixed_6b'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch_1 = slim.conv2d(net, 128, [1, 1], scope='Conv2d_0a_1x1')
                branch_1 = slim.conv2d(branch_1, 128, [1, 7], scope='Conv2d_0b_1x7') # 串联1*7卷积和7*1卷积合成7*7卷积，减少了参数，减轻了过拟合
                branch_1 = slim.conv2d(branch_1, 192, [7, 1], scope='Conv2d_0c_7x1')
            with tf.variable_scope('Branch_2'):
                branch_2 = slim.conv2d(net, 128, [1, 1], scope='Conv2d_0a_1x1') # 反复将7*7卷积拆分
                branch_2 = slim.conv2d(branch_2, 128, [7, 1], scope='Conv2d_0b_7x1')
                branch_2 = slim.conv2d(branch_2, 128, [1, 7], scope='Conv2d_0c_1x7')
                branch_2 = slim.conv2d(branch_2, 128, [7, 1], scope='Conv2d_0d_7x1')
                branch_2 = slim.conv2d(branch_2, 192, [1, 7], scope='Conv2d_0e_1x7')
            with tf.variable_scope('Branch_3'):
                branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
                branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_0b_1x1')
            net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
        # 192+192+192+192 = 768
        # mixed4: 17 x 17 x 768.


        with tf.variable_scope('Mixed_6c'):
            with tf.variable_scope('Branch_0'):
                '''
                我们的网络每经过一个inception module，即使输出尺寸不变，但是特征都相当于被重新精炼了一遍，
                其中丰富的卷积和非线性化对提升网络性能帮助很大。
                '''
                branch_0 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch_1 = slim.conv2d(net, 160, [1, 1], scope='Conv2d_0a_1x1')
                branch_1 = slim.conv2d(branch_1, 160, [1, 7], scope='Conv2d_0b_1x7')
                branch_1 = slim.conv2d(branch_1, 192, [7, 1], scope='Conv2d_0c_7x1')
            with tf.variable_scope('Branch_2'):
                branch_2 = slim.conv2d(net, 160, [1, 1], scope='Conv2d_0a_1x1')
                branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_0b_7x1')
                branch_2 = slim.conv2d(branch_2, 160, [1, 7], scope='Conv2d_0c_1x7')
                branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_0d_7x1')
                branch_2 = slim.conv2d(branch_2, 192, [1, 7], scope='Conv2d_0e_1x7')
            with tf.variable_scope('Branch_3'):
                branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
                branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_0b_1x1')
            net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
          # 192+192+192+192 = 768
          # mixed_5: 17 x 17 x 768.


        with tf.variable_scope('Mixed_6d'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch_1 = slim.conv2d(net, 160, [1, 1], scope='Conv2d_0a_1x1')
                branch_1 = slim.conv2d(branch_1, 160, [1, 7], scope='Conv2d_0b_1x7')
                branch_1 = slim.conv2d(branch_1, 192, [7, 1], scope='Conv2d_0c_7x1')
            with tf.variable_scope('Branch_2'):
                branch_2 = slim.conv2d(net, 160, [1, 1], scope='Conv2d_0a_1x1')
                branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_0b_7x1')
                branch_2 = slim.conv2d(branch_2, 160, [1, 7], scope='Conv2d_0c_1x7')
                branch_2 = slim.conv2d(branch_2, 160, [7, 1], scope='Conv2d_0d_7x1')
                branch_2 = slim.conv2d(branch_2, 192, [1, 7], scope='Conv2d_0e_1x7')
            with tf.variable_scope('Branch_3'):
                branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
                branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_0b_1x1')
            net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
          # 92+192+192+192 = 768
          # mixed_6: 17 x 17 x 768.

        with tf.variable_scope('Mixed_6e'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch_1 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_0a_1x1')
                branch_1 = slim.conv2d(branch_1, 192, [1, 7], scope='Conv2d_0b_1x7')
                branch_1 = slim.conv2d(branch_1, 192, [7, 1], scope='Conv2d_0c_7x1')
            with tf.variable_scope('Branch_2'):
                branch_2 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_0a_1x1')
                branch_2 = slim.conv2d(branch_2, 192, [7, 1], scope='Conv2d_0b_7x1')
                branch_2 = slim.conv2d(branch_2, 192, [1, 7], scope='Conv2d_0c_1x7')
                branch_2 = slim.conv2d(branch_2, 192, [7, 1], scope='Conv2d_0d_7x1')
                branch_2 = slim.conv2d(branch_2, 192, [1, 7], scope='Conv2d_0e_1x7')
            with tf.variable_scope('Branch_3'):
                branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
                branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_0b_1x1')
            net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
          # 92+192+192+192 = 768
          # mixed_7: 17 x 17 x 768.


            '''    
            第二个Inception组结束  一共五个Inception模块 输出为:17*17*768
    ----------------------------------------------------------------------    
            第三个Inception组   共3个Inception模块(带分支)
            '''
          # 将Mixed_6e存储于end_points中，作为Auxiliary Classifier辅助模型的分类
        end_points['Mixed_6e'] = net

          # 第三个inception模块组包含了三个inception module

        with tf.variable_scope('Mixed_7a'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_0a_1x1')# 17*17*192
                branch_0 = slim.conv2d(branch_0, 320, [3, 3], stride=2,
                                     padding='VALID', scope='Conv2d_1a_3x3') # 8*8*320
            with tf.variable_scope('Branch_1'):
                branch_1 = slim.conv2d(net, 192, [1, 1], scope='Conv2d_0a_1x1') #17*17*192
                branch_1 = slim.conv2d(branch_1, 192, [1, 7], scope='Conv2d_0b_1x7')
                branch_1 = slim.conv2d(branch_1, 192, [7, 1], scope='Conv2d_0c_7x1')
                branch_1 = slim.conv2d(branch_1, 192, [3, 3], stride=2,
                                     padding='VALID', scope='Conv2d_1a_3x3') #8*8*192
            with tf.variable_scope('Branch_2'):
                branch_2 = slim.max_pool2d(net, [3, 3], stride=2, padding='VALID',
                                         scope='MaxPool_1a_3x3')  #8*8*768
            net = tf.concat([branch_0, branch_1, branch_2], 3) # 输出图片尺寸被缩小，通道数增加，tensor的总size在持续下降中
          # 320+192+768 = 1280
          # mixed_8: 8 x 8 x 1280.


        with tf.variable_scope('Mixed_7b'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 320, [1, 1], scope='Conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch_1 = slim.conv2d(net, 384, [1, 1], scope='Conv2d_0a_1x1')
                branch_1 = tf.concat([
                  slim.conv2d(branch_1, 384, [1, 3], scope='Conv2d_0b_1x3'),
                  slim.conv2d(branch_1, 384, [3, 1], scope='Conv2d_0b_3x1')], 3)
            with tf.variable_scope('Branch_2'):
                branch_2 = slim.conv2d(net, 448, [1, 1], scope='Conv2d_0a_1x1')
                branch_2 = slim.conv2d(branch_2, 384, [3, 3], scope='Conv2d_0b_3x3')
                branch_2 = tf.concat([
                  slim.conv2d(branch_2, 384, [1, 3], scope='Conv2d_0c_1x3'),
                  slim.conv2d(branch_2, 384, [3, 1], scope='Conv2d_0d_3x1')], 3)
            with tf.variable_scope('Branch_3'):
                branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
                branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_0b_1x1')
            net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3) # 输出通道数增加到2048
          # 320+(384+384)+(384+384)+192 = 2048
          # mixed_9: 8 x 8 x 2048.


        with tf.variable_scope('Mixed_7c'):
            with tf.variable_scope('Branch_0'):
                branch_0 = slim.conv2d(net, 320, [1, 1], scope='Conv2d_0a_1x1')
            with tf.variable_scope('Branch_1'):
                branch_1 = slim.conv2d(net, 384, [1, 1], scope='Conv2d_0a_1x1')
                branch_1 = tf.concat([
                  slim.conv2d(branch_1, 384, [1, 3], scope='Conv2d_0b_1x3'),
                  slim.conv2d(branch_1, 384, [3, 1], scope='Conv2d_0c_3x1')], 3)
            with tf.variable_scope('Branch_2'):
                branch_2 = slim.conv2d(net, 448, [1, 1], scope='Conv2d_0a_1x1')
                branch_2 = slim.conv2d(branch_2, 384, [3, 3], scope='Conv2d_0b_3x3')
                branch_2 = tf.concat([
                  slim.conv2d(branch_2, 384, [1, 3], scope='Conv2d_0c_1x3'),
                  slim.conv2d(branch_2, 384, [3, 1], scope='Conv2d_0d_3x1')], 3)
            with tf.variable_scope('Branch_3'):
                branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
                branch_3 = slim.conv2d(branch_3, 192, [1, 1], scope='Conv2d_0b_1x1')
            net = tf.concat([branch_0, branch_1, branch_2, branch_3], 3)
          # 320+(384+384)+(384+384)+192 = 2048
          # mixed_10: 8 x 8 x 2048.

        return net, end_points
#Inception V3网络的核心部分，即卷积层部分就完成了



# V3最后部分
# 全局平均池化、Softmax和Auxiliary Logits
def inception_v3(inputs,
                 num_classes=10, # 最后需要分类的数量（比赛数据集的种类数）
                 is_training=True, # 标志是否为训练过程，只有在训练时Batch normalization和Dropout才会启用
                 dropout_keep_prob=0.8, # 节点保留比率
                 prediction_fn=slim.softmax, # 最后用来分类的函数
                 spatial_squeeze=True, # 参数标志是否对输出进行squeeze操作（去除维度数为1的维度，比如5*3*1转为5*3）
                 reuse=None, # 是否对网络和Variable进行重复使用
                 scope='InceptionV3'): # 包含函数默认参数的环境
    with tf.variable_scope(scope, 'InceptionV3', [inputs, num_classes], # 定义参数默认值
                         reuse=reuse) as scope:
        with slim.arg_scope([slim.batch_norm, slim.dropout], # 定义标志默认值
                            is_training=is_training):
          # 拿到最后一层的输出net和重要节点的字典表end_points
            net, end_points = inception_v3_base(inputs, scope=scope) # 用定义好的函数构筑整个网络的卷积部分
            nett=net
          # Auxiliary logits作为辅助分类的节点，对分类结果预测有很大帮助
    with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d],
                          stride=1, padding='SAME'): # 将卷积、最大池化、平均池化步长设置为1
        aux_logits = end_points['Mixed_6e'] # 通过end_points取到Mixed_6e
        # end_points['Mixed_6e']  --> 17x17x768
        with tf.variable_scope('AuxLogits'):
            aux_logits = slim.avg_pool2d(
                    aux_logits, [5, 5], stride=3, padding='VALID',
                    scope='AvgPool_1a_5x5') #5x5x768

            aux_logits = slim.conv2d(aux_logits, 128, [1, 1],
                                   scope='Conv2d_1b_1x1') #5x5x128

          # Shape of feature map before the final layer.
            aux_logits = slim.conv2d(
              aux_logits, 768, [5, 5],
              weights_initializer=trunc_normal(0.01),
              padding='VALID', scope='Conv2d_2a_5x5')  #1x1x768

            aux_logits = slim.conv2d(
              aux_logits, num_classes, [1, 1], activation_fn=None,
              normalizer_fn=None, weights_initializer=trunc_normal(0.001),
              scope='Conv2d_2b_1x1')  # 1*1*1000

        if spatial_squeeze: # tf.squeeze消除tensor中前两个为1的维度。
            aux_logits = tf.squeeze(aux_logits, [1, 2], name='SpatialSqueeze')
            end_points['AuxLogits'] = aux_logits # 最后将辅助分类节点的输出aux_logits储存到字典表end_points中

      # 处理正常的分类预测逻辑
      # Final pooling and prediction
    with tf.variable_scope('Logits'):
        # net --> 8x8x2048
        net = slim.avg_pool2d(net, [8, 8], padding='VALID',
                              scope='AvgPool_1a_8x8') #1x1x2048

        net = slim.dropout(net, keep_prob=dropout_keep_prob, scope='Dropout_1b')
        end_points['PreLogits'] = net

        # 激活函数和规范化函数设为空
#         logits = slim.conv2d(net, num_classes, [1, 1], activation_fn=None,
#                              normalizer_fn=None, scope='Conv2d_1c_1x1') # 1x1x1000
        logits = slim.fully_connected(net, num_classes, activation_fn=None,
                             normalizer_fn=None, scope='fc')
        w_variables = slim.get_model_variables()[-2]
        if spatial_squeeze: # tf.squeeze去除输出tensor中维度为1的节点
            logits = tf.squeeze(logits, [1, 2], name='SpatialSqueeze')

#     end_points['Logits'] = logits
#     end_points['Predictions'] = prediction_fn(logits, scope='Predictions') # Softmax对结果进行分类预测
    return logits , nett, w_variables # 最后返回logits和包含辅助节点的end_points


# height, width = 299, 299 # 图片尺寸

tf.reset_default_graph()
is_training=True

# Inputs
x = neural_net_image_input((299, 299, 3))
y = neural_net_label_input(10)

with slim.arg_scope(inception_v3_arg_scope()): # scope中包含了batch normalization默认参数，激活函数和参数初始化方式的默认值
    logits,nett,ww = inception_v3(x, is_training=is_training) # inception_v3中传入inputs获取里logits和end_points

logits = tf.identity(logits, name='logits')
last_net = tf.identity(nett, name='last_net')
last_w = tf.identity(ww, name='last_w')

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.005).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')   


In [ ]:
epochs=8
save_model_path = './model./model'
saver = tf.train.Saver()
save_loss=[]

with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for epoch in range(epochs):       
        for batch_i in range(n_batches):
            for batch_features, batch_labels in load_preprocess_training_batch(batch_i, batch_size):
                sess.run(optimizer, feed_dict={
                    x: batch_features,
                    y: batch_labels})
            if batch_i%10==0:
                print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
                save_loss.append(print_stats(sess, batch_features, batch_labels, cost, accuracy))
        print(epoch+1)
        save_path = saver.save(sess, save_model_path)

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


# Set batch size if not already set
try:
    if batch_size:
        pass
except NameError:
    batch_size = 64

save_model_path = './model./model'
n_samples = 4
top_n_predictions = 3

def test_model():
    """
    Test the saved model against the test dataset
    """

    loaded_graph = tf.Graph()

    with tf.Session(graph=loaded_graph) as sess:
        # Load model
        loader = tf.train.import_meta_graph(save_model_path + '.meta')
        loader.restore(sess, save_model_path)

        # Get Tensors from loaded model
        loaded_x = loaded_graph.get_tensor_by_name('x:0')
        loaded_y = loaded_graph.get_tensor_by_name('y:0')
        loaded_logits = loaded_graph.get_tensor_by_name('logits:0')
        loaded_acc = loaded_graph.get_tensor_by_name('accuracy:0')
        
        valid_acc = sess.run(loaded_acc, feed_dict={
        loaded_x: test_features,
        loaded_y: test_labels})
        print('Testing Accuracy: {}\n'.format(valid_acc))       
        
        random_test_features=test_features[:n_samples]
        random_test_labels=test_labels[:n_samples]

        random_test_predictions = sess.run(
                tf.nn.top_k(loaded_logits, top_n_predictions),
                feed_dict={loaded_x: random_test_features, loaded_y: random_test_labels})
        
        display_image_predictions(random_test_features, random_test_labels, random_test_predictions)

test_model()

In [ ]:
save_model_path = './model./model'
n_samples = 10

def CAM_model():
    """
    Test the saved model against the test dataset
    """

    loaded_graph = tf.Graph()

    with tf.Session(graph=loaded_graph) as sess:
        # Load model
        loader = tf.train.import_meta_graph(save_model_path + '.meta')
        loader.restore(sess, save_model_path)

        # Get Tensors from loaded model
        loaded_x = loaded_graph.get_tensor_by_name('x:0')
        loaded_y = loaded_graph.get_tensor_by_name('y:0')

        loaded_w = loaded_graph.get_tensor_by_name('last_w:0')
        loaded_net = loaded_graph.get_tensor_by_name('last_net:0')

            
        random_test_features=test_features[:n_samples]
        random_test_labels=test_labels[:n_samples]

        scores0 = sess.run(slim.softmax(loaded_logits),
            feed_dict={loaded_x: random_test_features, loaded_y: random_test_labels})
        print(sess.run(tf.equal(tf.argmax(scores0, 1), tf.argmax(random_test_labels, 1))))

        w=sess.run(loaded_w, feed_dict={
                    loaded_x: random_test_features,
                    loaded_y: random_test_labels})
        net=sess.run(loaded_net, feed_dict={
                    loaded_x: random_test_features,
                    loaded_y: random_test_labels})
        
        print(net.shape)
        print(w.shape)
        for ij in range(len(random_test_labels)):
            weights_LR = w
            activation_lastconv = np.array([net[ij]])
            weights_LR = weights_LR.T
            activation_lastconv = activation_lastconv.T
            
            topNum = 1 # generate heatmap for top X prediction results
            scores=scores0[ij]
            scoresMean = np.mean(scores, axis=0)
            ascending_order = np.argsort(scoresMean)
            IDX_category = ascending_order[::-1] # [::-1] to sort in descending order

            curCAMmapAll = py_returnCAMmap(activation_lastconv, weights_LR[[random_test_labels[ij].index(1)],:])
            for kk in range(topNum):
                curCAMmap_crops = curCAMmapAll[:,:,kk]
                curCAMmapLarge_crops = cv2.resize(curCAMmap_crops, (256,256))
                curHeatMap = cv2.resize(im2double(curCAMmapLarge_crops),(256,256)) # this line is not doing much
                curHeatMap = im2double(curHeatMap)
                curHeatMap = py_map2jpg(curHeatMap, None, 'jet')
                image = cv2.resize(random_test_features[ij], (256, 256))
                curHeatMap = im2double(image)*0.2+im2double(curHeatMap)*0.7

                cv2.imshow('', curHeatMap)
                cv2.waitKey(0)

CAM_model()